In [1]:
import numpy as np
import cv2

class HoughTransform:

    def __init__(self, theta_res=1, rho_res=1):
        self.theta_res = theta_res
        self.rho_res = rho_res

    def hough_lines_acc(self, edge_img):
        height, width = edge_img.shape
        max_rho = int(np.ceil(np.sqrt(height**2 + width**2)))
        num_thetas = int(180 / self.theta_res)
        num_rhos = int(max_rho * 2 / self.rho_res) + 1
        accumulator = np.zeros((num_rhos, num_thetas), dtype=np.uint8)
        thetas = np.deg2rad(np.arange(0, 180, self.theta_res))
        rhos = np.arange(-max_rho, max_rho+self.rho_res, self.rho_res)
        y_idxs, x_idxs = np.nonzero(edge_img)
        for i in range(len(x_idxs)):
            x = x_idxs[i]
            y = y_idxs[i]
            for j in range(len(thetas)):
                rho = int(round(x*np.cos(thetas[j]) + y*np.sin(thetas[j]))/self.rho_res) + num_rhos//2
                accumulator[rho, j] += 1
        return accumulator, thetas, rhos

    def hough_peaks(self, hough_acc, num_peaks, threshold):
        peaks = []
        acc_copy = hough_acc.copy()
        for i in range(num_peaks):
            peak = np.unravel_index(np.argmax(acc_copy), acc_copy.shape)
            if acc_copy[peak] < threshold:
                break
            peaks.append(peak)
            acc_copy[peak] = 0
        return peaks

    def hough_lines_draw(self, img, peaks, thetas, rhos):
        for i in range(len(peaks)):
            rho_idx, theta_idx = peaks[i]
            rho = rhos[rho_idx]
            theta = thetas[theta_idx]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        return img

   


In [2]:
# Load image
img = cv2.imread('Lenna.jpg')

# Convert image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detector
edges = cv2.Canny(gray, 50, 150, apertureSize=3)



In [3]:
# Create HoughTransform object
hough = HoughTransform()

# Get Hough accumulator matrix and theta/rho arrays
acc, theta, rho = hough.hough_lines_acc(edges)



In [4]:
# Get peak locations
peaks = hough.hough_peaks(acc, 10, 100)



In [5]:
# Draw lines corresponding to the peaks
lines = hough.hough_lines_draw(img, peaks, theta, rho)



In [ ]:
# Show the result
cv2.imshow('Hough Transform', lines)
cv2.waitKey(0)
cv2.destroyAllWindows()